## Description similarity with BERT
based on https://towardsdatascience.com/bert-for-measuring-text-similarity-eec91c6bf9e1

In [1]:
%pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 5.9 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.1/77.1 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 2.7 MB/s eta 0:00:0000:0100:03
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.8/33.8 MB 5.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.8/9.8 MB 5.9 MB/s eta 0:00:0000:0100:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 6.6 MB/s eta 0:00:00a 0:00:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 6.8 MB/s eta 0:00:00a 0:00:01m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 4.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 9.5 MB/s eta 0:00:00
     ━━━━━━

In [3]:
%pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 5.6 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 14.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 502.3/502.3 kB 9.8 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
import pickle
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [5]:
# games_df = pickle.load(open('../data/steam_games_clean.pkl', 'rb'))
games_df = pd.read_csv('../data/steam_metadata_flask.csv')

In [6]:
model = SentenceTransformer('bert-base-nli-mean-tokens')

In [8]:
sentence_embeddings = model.encode(games_df['description'])

In [9]:
cosine_similarity([sentence_embeddings[0]], sentence_embeddings[1:3])

array([[0.6985648 , 0.74429715]], dtype=float32)

In [12]:
sentence_embeddings.shape

(21435, 768)

In [13]:
bert_similarity_matrix = cosine_similarity(sentence_embeddings, sentence_embeddings)

In [14]:
bert_similarity_matrix

array([[0.99999994, 0.69856477, 0.744297  , ..., 0.64103043, 0.6781196 ,
        0.6255747 ],
       [0.69856477, 1.        , 0.68526125, ..., 0.6964793 , 0.5983877 ,
        0.6567241 ],
       [0.744297  , 0.68526125, 1.        , ..., 0.71334666, 0.6614725 ,
        0.73732096],
       ...,
       [0.64103043, 0.6964793 , 0.71334666, ..., 1.        , 0.82699096,
        0.85705304],
       [0.6781196 , 0.5983877 , 0.6614725 , ..., 0.82699096, 0.99999994,
        0.7982627 ],
       [0.6255747 , 0.6567241 , 0.73732096, ..., 0.85705304, 0.7982627 ,
        0.9999998 ]], dtype=float32)

In [15]:
pickle.dump(bert_similarity_matrix, open('../models/bert_similarity.pkl', 'wb'))

In [16]:
# Construct a reverse map of indices and game titles
indices = pd.Series(games_df.index, index=games_df['name'])

In [17]:
def recommend_content(title, sim_matrix):
    '''Get similar games based on content using the similarity matrix'''
    # get index for our game
    idx = indices[title]
    
    # get pairwise similarity scores of all games w.r.t to our game
    sim_scores = list(enumerate(sim_matrix[idx]))
    
    # sort scores based on similarity
    sorted_sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # make a dictionary with title as key and score as value
    content_similar_scores = {indices.index[i[0]]: i[1] for i in sorted_sim_scores[1:21]}

    return content_similar_scores

In [19]:
# test cosine similarity matrix
recommend_content('Cyberpunk 2077', bert_similarity_matrix)

{'LandTraveller': 0.8954308,
 'DOOM': 0.8907024,
 'Vertical Drop Heroes HD': 0.88395154,
 'Mechajammer': 0.8832306,
 'Shiness: The Lightning Kingdom': 0.88258225,
 'Draft of Darkness': 0.88142407,
 'Heroes of Hammerwatch': 0.87978274,
 'Hard Truck Apocalypse: Rise Of Clans / Ex Machina: Meridian 113': 0.87974846,
 'Wanderlust Adventures': 0.87886906,
 'Warhammer Underworlds: Online': 0.87861145,
 'League of Evil': 0.8771189,
 'CODE VEIN': 0.8768052,
 "Eliosi's Hunt": 0.8764241,
 'Reverse Crawl': 0.8762506,
 'Super Dungeon Tactics': 0.8753563,
 'Paper Sorcerer': 0.8750716,
 'HordeCore: Training Ground': 0.87471235,
 'Descent: Road to Legend': 0.8743857,
 'Dark Messiah of Might & Magic': 0.87411875,
 'Shatter': 0.87408954}